In [35]:
import duckdb
import pandas as pd

In [36]:
# Подключение к базе данных DuckDB
db_path = '../db/edfitter.duckdb'
conn = duckdb.connect(database=db_path, read_only=True)

In [37]:
# Загрузка данных из таблиц в Pandas DataFrame
courses_df = conn.execute("SELECT * FROM main.courses").fetchdf()
feedback_df = conn.execute("SELECT * FROM main.feedback").fetchdf()
users_df = conn.execute("SELECT * FROM main.users").fetchdf()
session_time_df = conn.execute("SELECT * FROM main.session_time").fetchdf()

In [38]:
# Объединение данных
# 1. Объединяем `courses` и `feedback` по `session_id`
courses_feedback_df = courses_df.merge(feedback_df, on='session_id', how='left', suffixes=('', '_feedback'))

# 2. Объединяем результат с `session_time` по `session_id`
courses_feedback_time_df = courses_feedback_df.merge(session_time_df, on='session_id', how='left', suffixes=('', '_time'))

# 3. Добавляем данные из `users` по `user_id`
final_df = courses_feedback_time_df.merge(users_df, on='user_id', how='left', suffixes=('', '_user'))


In [39]:
# Объединение столбцов user_id и id в один столбец
# final_df['user_id'] = final_df['user_id'].fillna(final_df['id'])
# Удаление лишней колонки 'id', которая больше не нужна
# final_df = final_df.drop(columns=['id'], errors='ignore')
# Удаление столбца `Уровень обучения`
final_df = final_df.drop(columns=['level'], errors='ignore')

In [40]:
final_df


,user_id,session_id,course_id,label,discipline,department,faculty,campus,audience,format,...,end_timestamp,elapsed_time_ms,elapsed_time_s,hours,minutes,seconds,milliseconds,username,group_number,role
0,0d097d3514c8c3fe,f6a05fcd3cf8edb9,6442798316,Бакалавриат,"Проектный семинар ""Основы программирования""",Кафедра информационных систем и технологий,"Факультет информатики, математики и компьютерн...",НИУ ВШЭ - Нижний Новгород,для своего кампуса,без онлайн-курса,...,1731433387.36300,73008.0,73.008003,0,1,13,8,Тест,12421,Студент (МБ по бизнесу и экономике)
1,0d097d3514c8c3fe,f6a05fcd3cf8edb9,16281501185,Бакалавриат,"Научно-исследовательский семинар ""Введение в с...",Кафедра информационных систем и технологий,"Факультет информатики, математики и компьютерн...",НИУ ВШЭ - Нижний Новгород,для своего кампуса,без онлайн-курса,...,1731433387.36300,73008.0,73.008003,0,1,13,8,Тест,12421,Студент (МБ по бизнесу и экономике)
2,0d097d3514c8c3fe,f6a05fcd3cf8edb9,16281505338,Бакалавриат,Разработка программного обеспечения,департамент информатики,Факультет Санкт-Петербургская школа физико-мат...,НИУ ВШЭ - Санкт-Петербург,для своего кампуса,без онлайн-курса,...,1731433387.36300,73008.0,73.008003,0,1,13,8,Тест,12421,Студент (МБ по бизнесу и экономике)
3,0d097d3514c8c3fe,f6a05fcd3cf8edb9,16281496280,Бакалавриат,Python программирование и обработка данных,Международный институт экономики и финансов,Международный институт экономики и финансов,Москва,для своего кампуса,без онлайн-курса,...,1731433387.36300,73008.0,73.008003,0,1,13,8,Тест,12421,Студент (МБ по бизнесу и экономике)
4,0d097d3514c8c3fe,f6a05fcd3cf8edb9,16281499914,Бакалавриат,Основы программирования и информатики,отдел сопровождения учебного процесса в бакала...,факультет экономических наук,Москва,для своего кампуса,без онлайн-курса,...,1731433387.36300,73008.0,73.008003,0,1,13,8,Тест,12421,Студент (МБ по бизнесу и экономике)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,a5e8fed2c94022c4,9c5aa777f8c0f6eb,16281496084,Магистратура,Политический менеджмент,департамент политики и управления,факультет социальных наук,Москва,для своего кампуса,без онлайн-курса,...,1731680945.07400,434646.0,434.645996,0,7,14,646,Лемешко Валентина Валерьевна,bps212,Другое
776,a5e8fed2c94022c4,9c5aa777f8c0f6eb,16281496311,Магистратура,Прикладное политическое письмо,департамент политики и управления,факультет социальных наук,Москва,для своего кампуса,без онлайн-курса,...,1731680945.07400,434646.0,434.645996,0,7,14,646,Лемешко Валентина Валерьевна,bps212,Другое
777,a5e8fed2c94022c4,9c5aa777f8c0f6eb,5509359349,"Бакалавриат, Специалитет, Магистратура, Аспира...",Политическая юриспруденция,департамент политики и управления,факультет социальных наук,Москва,для своего кампуса,без онлайн-курса,...,1731680945.07400,434646.0,434.645996,0,7,14,646,Лемешко Валентина Валерьевна,bps212,Другое
778,a5e8fed2c94022c4,9c5aa777f8c0f6eb,16281507451,"Бакалавриат, Специалитет, Магистратура, Аспира...",Введение в специальность,факультет химии,факультет химии,Москва,для своего кампуса,без онлайн-курса,...,1731680945.07400,434646.0,434.645996,0,7,14,646,Лемешко Валентина Валерьевна,bps212,Другое


In [41]:
# final_df.to_csv('../db/test.csv', index=False)

In [42]:
# Переименование и упорядочивание столбцов
final_df = final_df.rename(columns={
    'user_id': 'user_id',
    'username': 'user_name',
    'group_number': 'group_N',
    'role': 'role',
    'message': 'prompt',
    'session_id': 'prompt_id',
    'relevant_vacancy_skills': 'relevant_vacancy_skills',
    'unrelated_vacancy_skills': 'bad_vacancy_skills',
    'additional_vacancy_skills': 'new_vacancy_skills',
    'discipline': 'course_name',
    'course_id' : 'course_id',
    'label': 'level',
    'course_number': 'course_number',
    'department': 'department',
    'faculty': 'faculty',
    'campus': 'campus',
    'audience': 'audience',
    'format': 'format',
    'relevance': 'course_rel',
    'relevant_skills' : 'relevant_subjects_skills',
    'unrelated_skills' : 'bad_subjects_skills',
    'additional_subject_skills' : 'new_subjects_skills',
    'utility': 'app_useful',
    'popularity': 'app_demand',
    'comfort': 'app_handy',
    'feedback_comment': 'comments',
    'start_timestamp' : 'start_timestamp',
    'end_timestamp' : 'end_timestamp',
    'elapsed_time_s' : 'elapsed_time_s'
})

# Установка порядка столбцов в соответствии с `rename`
column_order = [
    'user_id', 'user_name', 'group_N', 'role', 'prompt', 'prompt_id', 
    'relevant_vacancy_skills', 'bad_vacancy_skills', 'new_vacancy_skills', 
    'course_name', 'course_id', 'level', 'course_number', 
    'department', 'faculty', 'campus', 'audience', 'format', 
    'course_rel', 'relevant_subjects_skills', 'bad_subjects_skills', 
    'new_subjects_skills', 'app_useful', 'app_demand', 'app_handy', 
    'comments', 'start_timestamp', 'end_timestamp', 'elapsed_time_s'
]

# Перестановка столбцов
final_df = final_df[column_order]
final_df.head(5) 
# Сохранение результата в CSV

,user_id,user_name,group_N,role,prompt,prompt_id,relevant_vacancy_skills,bad_vacancy_skills,new_vacancy_skills,course_name,...,relevant_subjects_skills,bad_subjects_skills,new_subjects_skills,app_useful,app_demand,app_handy,comments,start_timestamp,end_timestamp,elapsed_time_s
0,0d097d3514c8c3fe,Тест,12421,Студент (МБ по бизнесу и экономике),Хочу быть программистом,f6a05fcd3cf8edb9,Управление проектами; Техническая документация...,Организаторские навыки; Грамотная речь; Работа...,2552352; впвп ап; 36226; квпва; 626626,"Проектный семинар ""Основы программирования""",...,Множества; Словари; Ввод-вывод; Арифметические...,Python; Функциональное программирование; Объек...,124; 5325; 62626,6,3,5,впвпвпвп,1731433314.35500,1731433387.36300,73.008003
1,0d097d3514c8c3fe,Тест,12421,Студент (МБ по бизнесу и экономике),Хочу быть программистом,f6a05fcd3cf8edb9,Управление проектами; Техническая документация...,Организаторские навыки; Грамотная речь; Работа...,2552352; впвп ап; 36226; квпва; 626626,"Научно-исследовательский семинар ""Введение в с...",...,Bash; Perl; Python; Высшие структуры данных,Скриптовое программирование; Программистское м...,124; 5325; 62626,6,3,5,впвпвпвп,1731433314.35500,1731433387.36300,73.008003
2,0d097d3514c8c3fe,Тест,12421,Студент (МБ по бизнесу и экономике),Хочу быть программистом,f6a05fcd3cf8edb9,Управление проектами; Техническая документация...,Организаторские навыки; Грамотная речь; Работа...,2552352; впвп ап; 36226; квпва; 626626,Разработка программного обеспечения,...,Жизненный цикл по; Управление проектами; Uml; ...,Agile методологии; Scrum,124; 5325; 62626,6,3,5,впвпвпвп,1731433314.35500,1731433387.36300,73.008003
3,0d097d3514c8c3fe,Тест,12421,Студент (МБ по бизнесу и экономике),Хочу быть программистом,f6a05fcd3cf8edb9,Управление проектами; Техническая документация...,Организаторские навыки; Грамотная речь; Работа...,2552352; впвп ап; 36226; квпва; 626626,Python программирование и обработка данных,...,Python; Data structures; Loops; File handling;...,Conditional statements; Api queries,124; 5325; 62626,6,3,5,впвпвпвп,1731433314.35500,1731433387.36300,73.008003
4,0d097d3514c8c3fe,Тест,12421,Студент (МБ по бизнесу и экономике),Хочу быть программистом,f6a05fcd3cf8edb9,Управление проектами; Техническая документация...,Организаторские навыки; Грамотная речь; Работа...,2552352; впвп ап; 36226; квпва; 626626,Основы программирования и информатики,...,Java; Алгоритмы; Структуры данных; Дек; Очеред...,,124; 5325; 62626,6,3,5,впвпвпвп,1731433314.35500,1731433387.36300,73.008003


In [43]:
final_df.to_csv('../db/edfitter_data.csv', index=False)

In [44]:
final_df.to_excel('../db/edfitter_data.xlsx', index=False)

In [45]:
# Закрытие соединения
conn.close()